In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [53]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

In [55]:
q = 0
other_qs = list(range(translator.n_qubits))
other_qs.remove(q)
indds = [translator.cnots_index[str([q,k])] for k in other_qs]
cdb = pd.DataFrame([gate_template(i) for i in indds])

circuit_db = concatenate_dbs([z_layer_db(translator), cdb])

for q in range(1, translator.n_qubits):
    other_qs = list(range(translator.n_qubits))
    other_qs.remove(q)
    indds = [translator.cnots_index[str([q,k])] for k in other_qs]
    cdb = pd.DataFrame([gate_template(i) for i in indds])
    circuit_db = concatenate_dbs([circuit_db ,cdb])

c, circuit_db = translator.give_circuit(circuit_db)

db1 = concatenate_dbs([x_layer_db(translator)])
#db2 = concatenate_dbs([z_layer_db(translator)]*3)
#db3 = concatenate_dbs([x_layer_db(translator)]*3)
#db4 = concatenate_dbs([z_layer_db(translator)]*3)

circuit_db = concatenate_dbs([circuit_db, db1])
c, circuit_db = translator.give_circuit(circuit_db)

cdb = []
for ind, qubits in list(translator.indexed_cnots.items())[:3]:
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
c6_db = pd.DataFrame(cdb)
circuit_db = concatenate_dbs([c6_db, db1])

c, circuit_db = translator.give_circuit(circuit_db)

In [56]:
c

┌─────────┐              ┌─────────┐
(0, 0): ───@───Rz(th_0)────@────────────Rz(th_2)────@────────────Rz(th_4)───Rx(th_6)───
           │               │                        │
(0, 1): ───X───Rx(th_1)────┼Rx(th_7)────────────────┼──────────────────────────────────
                           │                        │
(0, 2): ───────────────────X────────────Rx(th_3)────┼Rx(th_8)──────────────────────────
                                                    │
(0, 3): ────────────────────────────────────────────X────────────Rx(th_5)───Rx(th_9)───
                          └─────────┘              └─────────┘

In [57]:
simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.absolute_rules = [simplifier.rule_4,simplifier.rule_5, simplifier.rule_6]
simplifier.relative_rules = [simplifier.rule_1, simplifier.rule_2]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>
warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_0
 symbols ['th_0', 'th_1']
circuit_db {0: {'ind': 16, 'symbol': 'th_0', 'param_value': 0.0, 'trainable': True, 'block_id': 0.0}, 1: {'ind': 17, 'symbol': 'th_1', 'param_value': 0.0, 'trainable': True, 'block_id': 0.0}, 2: {'ind': 1, '

AttributeError: ojo <bound method Simplifier.rule_1 of <utilities.simplifier.Simplifier object at 0x7f2d77d6c080>>

In [24]:
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(c, circuit_db)
simplified_db = circuit_db.copy()

In [29]:
simplification = False
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
        ## if i have a rotation and then a CNOT
        if (check_rot(ind_gate, simplifier.translator) is True) and (check_cnot(ind_gate_p1, simplifier.translator) is True):
            type_0 = type_get(ind_gate, simplifier.translator)
            control, target = simplifier.translator.indexed_cnots[str(ind_gate_p1)]

            this_qubit = q
            other_qubits = [control, target]
            other_qubits.remove(q)
            other_qubit = other_qubits[0]

            ### now it happens two interesting things: type0 == 0 AND q == control
            ### or type_0 == 1 AND q == target  then swap orders
            if ((type_0 == 0) and (q == control)) or ((type_0 == 1) and (q == target)):
                if len(on_qubit_order[q]) <2:
                    simplification=False
                else:
                    simplification = True
                    my_info = q, order_gate_on_qubit
                    ###now we swap the order in which we apply the rotation and the CNOT.
                    index_rot = on_qubit_order[q][order_gate_on_qubit]
                    info_rot = simplified_db.loc[index_rot].copy()
                    break

In [39]:
q, order_gate_on_qubit = my_info

In [32]:
my_info

(0, 1)

In [ ]:
simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#

simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1 ] + 0.1] = info_rot
simplified_db = simplified_db.sort_index().reset_index(drop=True)
simplified_db = order_symbol_labels(simplified_db)

In [36]:
simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#

In [40]:
simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1 ] + 0.1] = info_rot

In [45]:
simplified_db1 = simplified_db.sort_index().reset_index(drop=True)


In [48]:
ss = order_symbol_labels(simplified_db1)

In [49]:
ss

,ind,symbol,param_value,trainable,block_id
0,0,None,NaN,True,0
1,17,th_0,0.183191,True,0
2,1,None,NaN,True,0
3,12,th_1,0.289406,True,0
4,12,th_2,0.586513,True,0
5,18,th_3,0.020108,True,0
6,2,None,NaN,True,0
7,12,th_4,0.828940,True,0
8,19,th_5,0.004695,True,0
9,16,th_6,0.000000,True,0


In [52]:
check_symbols_ordered(simplified_db)

False

In [51]:
circuit_db

,ind,symbol,param_value,trainable,block_id
0,0,None,NaN,True,0
1,12,th_0,0.289406,True,0
2,17,th_1,0.183191,True,0
3,1,None,NaN,True,0
4,12,th_2,0.586513,True,0
5,18,th_3,0.020108,True,0
6,2,None,NaN,True,0
7,12,th_4,0.828940,True,0
8,19,th_5,0.004695,True,0
9,16,th_6,0.000000,True,0


In [23]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)



0.1922517703919343
                          ┌─────────┐              ┌─────────┐
(0, 0): ───@───Rz(th_0)────@────────────Rz(th_2)────@────────────Rz(th_4)───Rx(th_6)───
           │               │                        │
(0, 1): ───X───Rx(th_1)────┼Rx(th_7)────────────────┼──────────────────────────────────
                           │                        │
(0, 2): ───────────────────X────────────Rx(th_3)────┼Rx(th_8)──────────────────────────
                                                    │
(0, 3): ────────────────────────────────────────────X────────────Rx(th_5)───Rx(th_9)───
                          └─────────┘              └─────────┘

                          ┌─────────┐   ┌─────────┐
(0, 0): ───Rx(th_0)───@────────────@─────────────@────Rz(th_6)───Rz(th_7)───Rz(th_8)───
                      │            │             │
(0, 1): ──────────────X────Rx(th_1)┼─────Rx(th_2)┼─────────────────────────────────────
                                   │             │
(0, 2): 

THE PROBLEM IS RULE 6 !!! 
come on!